In [44]:
key_words = ["a","about","above","after","again","against","all","am","an","and","any","are","aren't","as","at","be","because","been","before","being","below","between","both","but","by","can't","cannot","could","couldn't","did","didn't","do","does","doesn't","doing","don't","down","during","each","few","for","from","further","had","hadn't","has","hasn't","have","haven't","having","he","he'd","he'll","he's","her","here","here's","hers","herself","him","himself","his","how","how's","i","i'd","i'll","i'm","i've","if","in","into","is","isn't","it","it's","its","itself","let's","me","more","most","mustn't","my","myself","no","nor","not","of","off","on","once","only","or","other","ought","our","ours","ourselves","out","over","own","same","shan't","she","she'd","she'll","she's","should","shouldn't","so","some","such","than","that","that's","the","their","theirs","them","themselves","then","there","there's","these","they","they'd","they'll","they're","they've","this","those","through","to","too","under","until","up","very","was","wasn't","we","we'd","we'll","we're","we've","were","weren't","what","what's","when","when's","where","where's","which","while","who","who's","whom","why","why's","with","won't","would","wouldn't","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves"]

punctuation_pattern = r'[!\"#$%&\'()*+,-./:;<=>?@\[\]\^_`{|}~—]'

In [52]:
import pandas as pd
import re

def addRowsToDataframe(df, row):

    df.loc[len(df)] = row

def checkKeyWords(string):

    for keyword in key_words:
        match = re.search(rf'^{keyword}$', string=string)
        if(match):
            return False
    
    return True

def createHTSDict(path):
    df = pd.read_json(path)
    columns_df = df.columns.tolist()
    final_dict = {}
    main_numbers_pattern = re.compile(r'^[\d]{4}')
    start = True
    current_main_hts = ['']
    previous_main_hts = ['']

    for index,row in df.iterrows():

        current_main_hts = main_numbers_pattern.findall(row['htsno'])

        if start:
            previous_main_hts = current_main_hts
            final_dict[current_main_hts[0]] = pd.DataFrame(columns=columns_df)
            addRowsToDataframe(final_dict[current_main_hts[0]], row)

        if len(current_main_hts) == 0:
            current_main_hts = previous_main_hts

        if (previous_main_hts[0] != current_main_hts[0]) and (len(current_main_hts)):
            final_dict[current_main_hts[0]] = pd.DataFrame(columns=columns_df)
            addRowsToDataframe(final_dict[current_main_hts[0]], row)
            previous_main_hts = current_main_hts  
        elif start == False:
            addRowsToDataframe(final_dict[current_main_hts[0]], row)
        start = False

    return final_dict



In [53]:
HTS_dict = createHTSDict('../db_hts/chapter_1.json')
HTS_dict['0101']

,htsno,indent,description,superior,units,general,special,other,footnotes,quotaQuantity,additionalDuties,addiitionalDuties
0,0101,0,"Live horses, asses, mules and hinnies:",None,[],,,,[],,,NaN
1,,1,Horses:,true,[],,,,None,None,None,NaN
2,0101.21.00,2,Purebred breeding animals,None,[],Free,,Free,"[{'columns': ['general'], 'marker': '1', 'valu...",None,None,NaN
3,0101.21.00.10,3,Males,None,[No.],,,,[],None,None,NaN
4,0101.21.00.20,3,Females,None,[No.],,,,[],None,None,NaN
5,0101.29.00,2,Other,None,[],Free,,20%,"[{'columns': ['general'], 'marker': '1', 'valu...",None,None,NaN
6,0101.29.00.10,3,Imported for immediate slaughter,None,[No.],,,,[],None,None,NaN
7,0101.29.00.90,3,Other,None,[No.],,,,[],None,None,NaN
8,0101.30.00.00,1,Asses,None,[No.],6.8%,"Free (A+,AU,BH,CL,CO,D,E,IL,JO,KR,MA,OM,P,PA,P...",15%,"[{'columns': ['general'], 'marker': '1', 'valu...",None,None,NaN
9,0101.90,1,Other:,None,[],,,,[],,,NaN


In [49]:
string_dict = {}
file_dict = {}

for key,df in HTS_dict.items():
    
    string_dict[key] = []
    
    file_dict[key] = f'test_files/{key}.json'

    df.to_json(file_dict[key], orient='records')

    for row in df.iterrows():
        desc = re.sub(pattern=punctuation_pattern, repl='', string=row[1]['description']).lower()
        if(checkKeyWords(desc)):
            array_string = desc.split()
            if(len(array_string) > 1):
                for string in array_string:
                    string_dict[key].append(string)
            else: string_dict[key].append(desc)

string_dict

{'0101': ['live',
  'horses',
  'asses',
  'mules',
  'and',
  'hinnies',
  'horses',
  'purebred',
  'breeding',
  'animals',
  'males',
  'females',
  'imported',
  'for',
  'immediate',
  'slaughter',
  'asses',
  'imported',
  'for',
  'immediate',
  'slaughter'],
 '0102': ['live',
  'bovine',
  'animals',
  'cattle',
  'purebred',
  'breeding',
  'animals',
  'dairy',
  'male',
  'female',
  'male',
  'female',
  'cows',
  'imported',
  'specially',
  'for',
  'dairy',
  'purposes',
  'weighing',
  'less',
  'than',
  '90',
  'kg',
  'each',
  'weighing',
  '90',
  'kg',
  'or',
  'more',
  'each',
  'weighing',
  'less',
  'than',
  '90',
  'kg',
  'each',
  'male',
  'female',
  'weighing',
  '90',
  'kg',
  'or',
  'more',
  'but',
  'less',
  'than',
  '200',
  'kg',
  'each',
  'male',
  'female',
  'weighing',
  '200',
  'kg',
  'or',
  'more',
  'but',
  'less',
  'than',
  '320',
  'kg',
  'each',
  'male',
  'female',
  'weighing',
  '320',
  'kg',
  'or',
  'more',
  'ea